In [1]:
import pandas as pd #импорт данных
import nltk
from nltk.probability import FreqDist
import numpy as np
import re
from nltk.stem import wordnet #
from nltk import pos_tag 
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer #мешок слов
from sklearn.feature_extraction.text import TfidfVectorizer # определить частоту слова
from sklearn.metrics import pairwise_distances #косинусное сходство
from sklearn.cluster import KMeans



In [ ]:
!python -m spacy download en_core_web_lg

In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
import spacy
import textacy
from numpy import dot
from numpy.linalg import norm

# Загружаем датасет

In [5]:
df = pd.read_csv('Gigantika_booking.csv')
df

,title,author,category,text
0,Алёша-дурачок,Андреев Леонид,Русская классика,\n(Посвящается Э. В. Готье) Впервые ув...
1,Анатэма,Андреев Леонид,Русская классика,\nТрагическое представление в семи картинах. ...
2,Ангелочек,Андреев Леонид,Русская классика,\nI Временами Сашке хотелось перестать дел...
3,Анфиса,Андреев Леонид,Русская классика,\nЛеонид Андреев. АнфисаДЕЙСТВУЮЩИЕ ЛИЦА: ...
4,Баргамот и Гараська,Андреев Леонид,Русская классика,"\nБыло бы несправедливо сказать, что природа о..."
...,...,...,...,...
7828,Нумизматы,"Эллин, Стенли",Детективы,Стенли Эллин. НумизматыСодержаниеFine HTMLPrin...
7829,Выройте себе могилу,"Эллин, Стенли",Детективы,Стенли Эллин. Выройте себе могилуСодержаниеFin...
7830,Мешок-убийца,"Эллсон, Хал",Детективы,Хал Эллсон. Мешок-убийцаСодержаниеFine HTMLPri...
7831,Сдается комната,"Эллсон, Хал",Детективы,Хал Эллсон. Сдается комнатаСодержаниеFine HTML...


In [16]:
df = pd.read_csv('Kniga-4.csv') #загружаем файл
df = df[['title','ru_title','author','genre','text']]
df


,title,ru_title,genre,text,Lemmatized,D2V_Listed,Listed,SemSim,author
0,The Valley of Fear,Долина Страха,"Novel, Detective, Romance, Detective, Adventur...","PART 1 — The Tragedy of Birlstone\r\n,,,\r\nCh...",part 1 tragedy birlstone chapter 1 warning inc...,2,5,7,Артур Конан Дойл
1,The Beasts of Tarzan,Тарзан и его звери,"Novel, Adventure","Chapter I: Kidnapped\r\n\r\n""The entire affair...",chapter kidnap entire affair shroud mystery sa...,2,5,1,Эдгар Райс Берроуз
2,Kim,Ким,"Realism, novel, Adventure",Chapter 1\r\n\r\nOh ye who tread the Narrow Wa...,chapter 1 oh ye tread narrow way tophet flare ...,2,5,1,Редьярд Киплинг
3,At the Earth's Core,Необыкновенное путешествие к центру земли,"Fantastic, novel, Adventure",Prologue\r\n\r\nIN THE FIRST PLACE PLEASE BEAR...,prologue first place please bear mind expect b...,2,2,6,Эдгар Райс Берроуз
4,The Luck of Barry Lyndon,Удача Барри Линдона,Novel,Introduction\r\n\r\nA Bibliographical Note:\r\...,introduction bibliographical note far best kno...,0,5,1,Уильям Мейкпис Теккерей
...,...,...,...,...,...,...,...,...,...
205,The Man Who Laughs,"Человек, который смеётся",Роман,PRELIMINARY CHAPTER\n\nI. URSUS\n\nI\n\nURSUS ...,preliminary homo fast friend ursus man homo wo...,0,2,5,Виктор Гюго
206,The Gift,Дар,Роман,\nForeword\n\nTHE greater part of The Gift (in...,foreword great part gift russian dar write ber...,0,5,6,Владимир Набоков
207,Hearts of Three,Сердца трёх,Роман,\nFOREWORD\n\nI HOPE the reader will forgive m...,foreword hope reader forgive beginning forewor...,0,5,1,Джек Лондон
208,The Sea-Wolf,Морской Волк,Роман,"Chapter 1\n\nI scarcely know where to begin, t...",chapter 1 scarcely know begin though sometimes...,0,2,1,Джек Лондон


# Токенизация с помощью NLTK и Spacy

In [8]:
# Загрузка английской NLP-модели Spacy
nlp = spacy.load('en_core_web_lg')

# Проверка всех сущностей, удаление лишних слов
def scrub(text):
  no_names_text = []
  doc = nlp(text)
  for token in doc:
    if token.ent_type_ == "PERSON":
      continue
    elif token.ent_type_ == "FAC":
      continue
    elif token.ent_type_ == "ORG":
      continue
    elif token.ent_type_ == "GPE":
      continue
    elif token.ent_type_ == "PRODUCT":
      continue
    elif token.ent_type_ == "MONEY":
      continue
    elif token.ent_type_ == "LOC":
      continue
    elif token.ent_type_ == "DATE":
      continue
    else: 
      no_names_text.append(str(token))      
  return " ".join(no_names_text)

In [9]:
def text_normalization(text):
    text = str(text).lower() #занижаем регистр каждой строки
    spl_char = re.sub(r'[^a-z0-9]', ' ', text) #убираем лишние символы
    stop = stopwords.words('english')
    
    clear_tokens=[]
    for token in spl_char.split():
        if token.lower() not in stop:
            clear_tokens.append(token)
    j = ' '.join(clear_tokens)
    tokens = word_tokenize(j) #делаем токенизацию
    lemma = wordnet.WordNetLemmatizer() #обзываем лемматизацию
    tags_list = pos_tag(tokens, tagset= None) #определяем часть речи
    lemma_words = []
    for tokens, pos_token in tags_list: # первый элемент попадает в токен, а второй в постокен
        if pos_token.startswith('V'): #он делает лемматизацию, опираясь на часть речи
            pos_val = 'v'
        elif pos_token.startswith('J'):
            pos_val = 'a'
        elif pos_token.startswith('R'):
            pos_val = 'r'
        else:
            pos_val = 'n'
        lemma_token = lemma.lemmatize(tokens, pos_val)
        lemma_words.append(lemma_token)
    text_with_names = ' '.join(lemma_words)

    # Библиотека Spacy не поддерживает обработку текста длиннее 1млн символов, поэтому подобные тексты обрабатываем по частям
    if len(text_with_names) > 900000:  
      tokens = word_tokenize(text_with_names)
      first_part = []
      second_part = []
      third_part = []
      len_text = 0
      for word in tokens: 
          if len_text < len(tokens)//3 : # набираем первую половину текста в отдельный список
              first_part.append(word)
              len_text += 1
          elif len_text < len(tokens)//3*2  and len_text >= len(tokens)//3 :
              second_part.append(word)
              len_text += 1
          elif len_text >= len(tokens)//3*2 :
              third_part.append(word)
              len_text += 1
      first_part_text = ' '.join(first_part)
      second_part_text = ' '.join(second_part)
      third_part_text = ' '.join(third_part)
      doc = scrub(first_part_text) + ' ' + scrub(second_part_text) + ' ' + scrub(third_part_text)
    else: 
      doc = scrub(text_with_names)
    
    return doc
    # в переменной 'doc' теперь содержится обработанная версия текста

In [10]:
df['Lemmatized'] = df['text'].apply(text_normalization) # Создаём столбец с обработанным текстом
df['Lemmatized']

0     part 1 tragedy birlstone chapter 1 warning inc...
1     chapter kidnap entire affair shroud mystery sa...
2     chapter 1 oh ye tread narrow way tophet flare ...
3     prologue first place please bear mind expect b...
4     introduction bibliographical note far best kno...
                            ...                        
26    preliminary homo fast friend ursus man homo wo...
27    foreword great part gift russian dar write ber...
28    foreword hope reader forgive beginning forewor...
29    chapter 1 scarcely know begin though sometimes...
30    chapter one open door latch key go follow youn...
Name: Lemmatized, Length: 210, dtype: object

# D2V

In [11]:
common_texts = []
text_vectors = []
# Делаем один большой текст, объединяя все книги
def preparation_of_doc(check):
    a = check.split()
    common_texts.append(a)
df['Lemmatized'].apply(preparation_of_doc)
# обучаем модель
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=200, window=1, min_count=1, workers=4, epochs=50)
fname = get_tmpfile("my_doc2vec_model")
model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

# преобразуем текст каждой книги в векторы
def D2V_text(text):
  vector = model.infer_vector(text.split())
  text_vectors.append(vector)
df['Lemmatized'].apply(D2V_text) 
df_d2v = pd.DataFrame(text_vectors)
df_d2v.to_csv('D2V_results_rus.csv', index=False)
df_d2v

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.999619,-0.808159,-0.994674,1.918554,0.557592,-0.832124,-0.238977,-1.773246,-2.003402,2.070271,...,-1.251926,2.014437,0.834616,0.352908,-0.919878,1.622829,1.499665,-0.835192,0.510972,1.755962
1,-2.474602,-2.044590,0.843441,-1.742882,1.147414,-2.378116,-0.441220,-0.890201,-0.693671,-0.941824,...,-1.037325,0.428693,-0.498527,-0.351029,-1.020584,0.166774,0.650675,2.306175,0.799053,-0.138005
2,-2.977500,-3.185117,0.327249,0.309487,0.782983,0.317552,0.146557,-1.310871,1.893729,2.508066,...,-1.000512,-0.128233,-1.970839,-1.631990,-2.039807,-0.103885,0.891152,-0.228723,0.893996,1.849702
3,-3.306579,-3.026248,-1.267522,-1.442858,-0.458798,0.192212,-1.350327,-1.228954,-1.112000,-0.846444,...,-2.422247,0.829607,0.136057,-0.881232,-1.494303,0.717322,1.161820,-0.597066,2.610999,0.656048
4,-2.645420,-2.235759,-0.300125,0.947038,-0.528068,0.875328,-0.992479,0.811358,0.065952,0.481380,...,-2.031230,-1.950923,0.000549,0.743743,-0.633630,1.772366,-1.428726,-0.596074,-0.293725,1.173366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,-2.458881,-1.137864,-0.563854,-0.892880,2.016692,0.713697,1.256184,0.783463,-0.446947,-2.206580,...,-2.063841,0.325260,-1.139585,0.961121,1.079422,2.170224,-0.575176,-1.283815,0.686007,0.449874
206,-1.244406,-2.301752,-0.138228,-0.880284,0.741000,-1.449641,-2.122716,-1.211166,-3.135102,0.958424,...,-3.352870,1.114873,0.307087,0.529647,-0.251589,-1.312794,-0.895182,-0.412754,0.748703,1.214520
207,-0.477682,-3.010777,-0.196687,-1.200302,-0.066370,-1.549905,-1.697617,-0.086183,1.072135,1.746599,...,-2.217743,0.449632,-0.596099,0.759448,-2.452349,-0.311331,-0.151113,0.307088,0.640556,0.934239
208,-1.227344,-0.121065,-0.760411,-0.811175,0.828324,0.104213,1.543100,-0.188936,0.789180,-0.029126,...,-0.382064,1.168294,-2.989446,-0.224803,-2.543634,1.718924,0.126834,0.235492,0.053167,0.789967


In [12]:
# Проводим кластеризацию без учителя
model_d2v = KMeans(n_clusters=10) #указываем желаемое кол-во кластеров
# Проводим моделирование
model_d2v.fit(text_vectors)
# Предсказание на всем наборе данных
d2v_predictions = model_d2v.predict(text_vectors)
# Добавляем столбец с кластеризацией по Doc2Vec
df['D2V_Listed'] = pd.DataFrame(d2v_predictions)
df

,title,ru_title,author,genre,text,Lemmatized,D2V_Listed
0,The Valley of Fear,Долина Страха,Arthur Conan Doyle,"Novel, Detective, Romance, Detective, Adventur...","PART 1 — The Tragedy of Birlstone\r\n,,,\r\nCh...",part 1 tragedy birlstone chapter 1 warning inc...,2
1,The Beasts of Tarzan,Тарзан и его звери,Edgar Rice Burroughs,"Novel, Adventure","Chapter I: Kidnapped\r\n\r\n""The entire affair...",chapter kidnap entire affair shroud mystery sa...,2
2,Kim,Ким,Joseph Rudyard Kipling,"Realism, novel, Adventure",Chapter 1\r\n\r\nOh ye who tread the Narrow Wa...,chapter 1 oh ye tread narrow way tophet flare ...,2
3,At the Earth's Core,Необыкновенное путешествие к центру земли,Edgar Rice Burroughs,"Fantastic, novel, Adventure",Prologue\r\n\r\nIN THE FIRST PLACE PLEASE BEAR...,prologue first place please bear mind expect b...,2
4,The Luck of Barry Lyndon,Удача Барри Линдона,William Makepeace Thackeray,Novel,Introduction\r\n\r\nA Bibliographical Note:\r\...,introduction bibliographical note far best kno...,0
...,...,...,...,...,...,...,...
26,The Man Who Laughs,"Человек, который смеётся",Виктор Гюго,Роман,PRELIMINARY CHAPTER\n\nI. URSUS\n\nI\n\nURSUS ...,preliminary homo fast friend ursus man homo wo...,0
27,The Gift,Дар,Владимир Набоков,Роман,\nForeword\n\nTHE greater part of The Gift (in...,foreword great part gift russian dar write ber...,0
28,Hearts of Three,Сердца трёх,Джек Лондон,Роман,\nFOREWORD\n\nI HOPE the reader will forgive m...,foreword hope reader forgive beginning forewor...,0
29,The Sea-Wolf,Морской Волк,Джек Лондон,Роман,"Chapter 1\n\nI scarcely know where to begin, t...",chapter 1 scarcely know begin though sometimes...,0


# k-кластеризация по соотношению частей речи в начале\середине\конце каждого текста

In [13]:
tagged_texts = []
def tegging(text):   
    str_tag = '' 
    text = str(text).lower() #занижаем регистр каждой строки
    spl_char = re.sub(r'[^a-z0-9]', ' ', text) #убираем лишние символы
    stop = stopwords.words('english') # убираем стоп-слова
    clear_tokens = []
    for token in spl_char.split():
        if token.lower() not in stop:
            clear_tokens.append(token)
    j = ' '.join(clear_tokens)
    tokens = word_tokenize(j) #делаем токенизацию
    tags_list = pos_tag(tokens, tagset= None) #определяем часть речи
    v1,v2,v3,j1,j2,j3,r1,r2,r3,n1,n2,n3 = 0,0,0,0,0,0,0,0,0,0,0,0
    len_text = 0     
    for tokens, pos_token in tags_list: # считаем количество существительных, прилагательных, глаголов, наречий
        if len_text < len(tags_list)//4:
            if pos_token.startswith('V'): # в первой трети текста
                v1 +=1
            elif pos_token.startswith('J'):
                j1 +=1
            elif pos_token.startswith('R'):
                r1 +=1
            else:
                n1 +=1
            len_text += 1
        if len_text >= len(tags_list)//4 and len_text < len(tags_list)//4*3:
            if pos_token.startswith('V'): # во второй трети текста
                v2 +=1
            elif pos_token.startswith('J'):
                j2 +=1
            elif pos_token.startswith('R'):
                r2 +=1
            else:
                n2 +=1
            len_text += 1
        if len_text >= len(tags_list)//4*3:
            if pos_token.startswith('V'): # в последней трети текста
                v3 +=1
            elif pos_token.startswith('J'):
                j3 +=1
            elif pos_token.startswith('R'):
                r3 +=1
            else:
                n3 +=1
    # Находим соотношение частей речи
    tagging_text = [(j1/v1),(r1/v1),(n1/v1),(j2/v2),(r2/v2),(n2/v2),(j3/v3),(r3/v3),(n3/v3)]
    tagged_texts.append(tagging_text)
    


In [14]:
df['text'].apply(tegging)



0     None
1     None
2     None
3     None
4     None
      ... 
26    None
27    None
28    None
29    None
30    None
Name: text, Length: 210, dtype: object

In [15]:
# добавляем столбец с распределением по частям речи
df_tag = pd.DataFrame(tagged_texts)
df_tag.to_csv('Tag_results_rus.csv', index=False)
df_tag.head(30)

,0,1,2,3,4,5,6,7,8
0,0.798851,0.325670,2.060664,0.775733,0.335933,1.976918,0.674985,0.293523,1.928105
1,0.814355,0.342148,2.032920,0.804749,0.323219,1.968602,0.743271,0.312629,1.964803
2,0.837780,0.305640,2.009703,0.809165,0.304191,2.015978,0.811902,0.303828,1.981160
3,0.795518,0.335434,1.932073,0.776511,0.332988,1.898791,0.791781,0.343836,1.841781
4,0.829130,0.307453,2.007934,0.846844,0.286624,2.101042,0.857266,0.288062,2.073241
5,0.683258,0.421317,1.747612,0.702640,0.371118,1.892081,0.673369,0.385974,1.700834
6,0.814415,0.332577,2.057321,0.754116,0.314440,1.930364,0.725187,0.319104,1.910352
7,0.883197,0.374388,2.052692,0.848170,0.367887,1.943251,0.807345,0.343760,1.961096
8,0.605759,0.338704,1.700720,0.645017,0.336554,1.823403,0.712505,0.334800,1.852952
9,0.901584,0.354072,1.919683,0.933449,0.343750,1.994792,0.838106,0.354075,1.875551


In [16]:
# проводим кластеризацию без учителя
model = KMeans(n_clusters=10)
# Проводим моделирование
model.fit(tagged_texts)
# Предсказание на всем наборе данных
all_predictions = model.predict(tagged_texts)
# добавляем столбец с распределением по частям речи
df['Listed'] = pd.DataFrame(all_predictions)
df

,title,ru_title,author,genre,text,Lemmatized,D2V_Listed,Listed
0,The Valley of Fear,Долина Страха,Arthur Conan Doyle,"Novel, Detective, Romance, Detective, Adventur...","PART 1 — The Tragedy of Birlstone\r\n,,,\r\nCh...",part 1 tragedy birlstone chapter 1 warning inc...,2,5
1,The Beasts of Tarzan,Тарзан и его звери,Edgar Rice Burroughs,"Novel, Adventure","Chapter I: Kidnapped\r\n\r\n""The entire affair...",chapter kidnap entire affair shroud mystery sa...,2,5
2,Kim,Ким,Joseph Rudyard Kipling,"Realism, novel, Adventure",Chapter 1\r\n\r\nOh ye who tread the Narrow Wa...,chapter 1 oh ye tread narrow way tophet flare ...,2,5
3,At the Earth's Core,Необыкновенное путешествие к центру земли,Edgar Rice Burroughs,"Fantastic, novel, Adventure",Prologue\r\n\r\nIN THE FIRST PLACE PLEASE BEAR...,prologue first place please bear mind expect b...,2,2
4,The Luck of Barry Lyndon,Удача Барри Линдона,William Makepeace Thackeray,Novel,Introduction\r\n\r\nA Bibliographical Note:\r\...,introduction bibliographical note far best kno...,0,5
...,...,...,...,...,...,...,...,...
26,The Man Who Laughs,"Человек, который смеётся",Виктор Гюго,Роман,PRELIMINARY CHAPTER\n\nI. URSUS\n\nI\n\nURSUS ...,preliminary homo fast friend ursus man homo wo...,0,2
27,The Gift,Дар,Владимир Набоков,Роман,\nForeword\n\nTHE greater part of The Gift (in...,foreword great part gift russian dar write ber...,0,5
28,Hearts of Three,Сердца трёх,Джек Лондон,Роман,\nFOREWORD\n\nI HOPE the reader will forgive m...,foreword hope reader forgive beginning forewor...,0,5
29,The Sea-Wolf,Морской Волк,Джек Лондон,Роман,"Chapter 1\n\nI scarcely know where to begin, t...",chapter 1 scarcely know begin though sometimes...,0,2


# Cемантическая близость

In [ ]:
textacy_list = []
bag_of_phrases = []
def textacy_work(text):
    # сокращаем длину текстов, превышающих лимит в 1 млн символов
    nlp = spacy.load('en_core_web_lg')
    if len(text) > 990000:
        print(len(text))
        len_text = 0
        cut_text = []
        split_text = text.split()
        print(len(split_text))
        for word in split_text:
            if len(" ".join(cut_text)) < 999990:
                cut_text.append(word)

        text1 = " ".join(cut_text)
        print(len(text1))
        doc = nlp(text1)
    else:
        doc = nlp(text)       
    # проводим поиск максмально важных словосочетаний
    doc = textacy.extract.keyterms.yake(doc, window_size = 5,topn = 300, ngrams= (2))
    textacy_list.append(doc)
    for phrases, coficient in doc:
        bag_of_phrases.append(phrases)
df['Lemmatized'].apply(textacy_work)


In [18]:
# присваеваем каждой фразе уникальный id 
dist = FreqDist(bag_of_phrases)
freq_phrases = dist.most_common()
num_phrases = []
for phrases, frequency in freq_phrases:
        if frequency > 1: 
            num_phrases.append(phrases)
all_num_phrases = list(enumerate(num_phrases))
all_books_num_phrases = []
#  удаляем фразы, что уникальны только для одной книги
for book_phrases in textacy_list:
    one_book_phrases = []
    one_book_clear_phrases = []
    for phrase, frequency in book_phrases:
        for number, com_word  in all_num_phrases:
            if phrase == com_word:
                one_book_phrases.append(number)
    all_books_num_phrases.append(one_book_phrases)
min_len_book = len(min(all_books_num_phrases, key=len))

top_text_all_book = []
for book in all_books_num_phrases:
    str_sim = ''
    top_text_one_book = []
    for text in book:
        if len(top_text_one_book) < min_len_book:
            top_text_one_book.append(text)
    top_text_all_book.append(top_text_one_book)
all_books_num_phrases = []
df_semsim = pd.DataFrame(top_text_all_book)
df_semsim.to_csv('SemSim_results_rus.csv', index= False)
df_semsim

,0,1,2,3,4,5,6,7,8,9
0,2495,2,1459,2496,2497,134,1460,29,45,12
1,1480,113,177,108,13,2522,3,4,55,242
2,1,518,624,318,113,20,4,12,244,0
3,2575,2576,2577,1,2578,2579,437,315,1515,2580
4,2,90,10,91,77,2786,11,20,70,320
...,...,...,...,...,...,...,...,...,...,...
205,1,4312,16,4,9,969,6,0,55,25
206,1,0,2,4,5189,5,384,24,299,988
207,1,2,31,4890,13,23,3,920,5557,0
208,1671,4615,2803,556,45,345,14,949,1481,3


In [19]:
# Проводим кластеризацию по семантической близости
model_for_phrases = KMeans(n_clusters=10)
model_for_phrases.fit(top_text_all_book)
phrases_predictions = model_for_phrases.predict(top_text_all_book)
df['SemSim'] = pd.DataFrame(phrases_predictions)
df.head()

,title,ru_title,author,genre,text,Lemmatized,D2V_Listed,Listed,SemSim
0,The Valley of Fear,Долина Страха,Arthur Conan Doyle,"Novel, Detective, Romance, Detective, Adventur...","PART 1 — The Tragedy of Birlstone\r\n,,,\r\nCh...",part 1 tragedy birlstone chapter 1 warning inc...,2,5,7
1,The Beasts of Tarzan,Тарзан и его звери,Edgar Rice Burroughs,"Novel, Adventure","Chapter I: Kidnapped\r\n\r\n""The entire affair...",chapter kidnap entire affair shroud mystery sa...,2,5,1
2,Kim,Ким,Joseph Rudyard Kipling,"Realism, novel, Adventure",Chapter 1\r\n\r\nOh ye who tread the Narrow Wa...,chapter 1 oh ye tread narrow way tophet flare ...,2,5,1
3,At the Earth's Core,Необыкновенное путешествие к центру земли,Edgar Rice Burroughs,"Fantastic, novel, Adventure",Prologue\r\n\r\nIN THE FIRST PLACE PLEASE BEAR...,prologue first place please bear mind expect b...,2,2,6
4,The Luck of Barry Lyndon,Удача Барри Линдона,William Makepeace Thackeray,Novel,Introduction\r\n\r\nA Bibliographical Note:\r\...,introduction bibliographical note far best kno...,0,5,1


# Сохранение полученных данных для дальнейшей работы

In [20]:
df.to_csv('Kniga-52.csv', index=False)

In [21]:

df = pd.read_csv('Kniga-52.csv') #загружаем файл
df


,title,ru_title,author,genre,text,Lemmatized,D2V_Listed,Listed,SemSim
0,The Valley of Fear,Долина Страха,Arthur Conan Doyle,"Novel, Detective, Romance, Detective, Adventur...","PART 1 — The Tragedy of Birlstone\r\n,,,\r\nCh...",part 1 tragedy birlstone chapter 1 warning inc...,2,5,7
1,The Beasts of Tarzan,Тарзан и его звери,Edgar Rice Burroughs,"Novel, Adventure","Chapter I: Kidnapped\r\n\r\n""The entire affair...",chapter kidnap entire affair shroud mystery sa...,2,5,1
2,Kim,Ким,Joseph Rudyard Kipling,"Realism, novel, Adventure",Chapter 1\r\n\r\nOh ye who tread the Narrow Wa...,chapter 1 oh ye tread narrow way tophet flare ...,2,5,1
3,At the Earth's Core,Необыкновенное путешествие к центру земли,Edgar Rice Burroughs,"Fantastic, novel, Adventure",Prologue\r\n\r\nIN THE FIRST PLACE PLEASE BEAR...,prologue first place please bear mind expect b...,2,2,6
4,The Luck of Barry Lyndon,Удача Барри Линдона,William Makepeace Thackeray,Novel,Introduction\r\n\r\nA Bibliographical Note:\r\...,introduction bibliographical note far best kno...,0,5,1
...,...,...,...,...,...,...,...,...,...
205,The Man Who Laughs,"Человек, который смеётся",Виктор Гюго,Роман,PRELIMINARY CHAPTER\n\nI. URSUS\n\nI\n\nURSUS ...,preliminary homo fast friend ursus man homo wo...,0,2,5
206,The Gift,Дар,Владимир Набоков,Роман,\nForeword\n\nTHE greater part of The Gift (in...,foreword great part gift russian dar write ber...,0,5,6
207,Hearts of Three,Сердца трёх,Джек Лондон,Роман,\nFOREWORD\n\nI HOPE the reader will forgive m...,foreword hope reader forgive beginning forewor...,0,5,1
208,The Sea-Wolf,Морской Волк,Джек Лондон,Роман,"Chapter 1\n\nI scarcely know where to begin, t...",chapter 1 scarcely know begin though sometimes...,0,2,1


# Определяем интересы пользователя

In [8]:
# Книга 1
title_1 = input('Please, call you name your first favorite book? \n') # ПРОВЕРИТЬ КАТРИОНУ
title_2 = input('Please, call you name your second favorite book? \n')
title_3 = input('Please, call you name your third favorite book? \n')
liked_book_1 = df[df['title'] == title_1]
liked_book_2 = df[df['title'] == title_2]
liked_book_3 = df[df['title'] == title_3]
ID_book_1 = df.loc[df['title']==title_1].index[0]
ID_book_2 = df.loc[df['title']==title_2].index[0]
ID_book_3 = df.loc[df['title']==title_3].index[0]
if liked_book_1.empty:
    liked_book_1 = df[df['ru_title'] == title_1]
    ID_book_1 = df.loc[df['ru_title']==title_1].index[0]
if liked_book_1.empty:
    print("I don`t know first book name, try to check the correctness \n")
    

if liked_book_2.empty:
    liked_book_2 = df[df['ru_title'] == title_2]
    ID_book_2 = df.loc[df['ru_title']==title_2].index[0]
if liked_book_2.empty:
    print("I don`t know second book name, try to check the correctness \n")

if liked_book_3.empty:
    liked_book_3 = df[df['ru_title'] == title_3]
    ID_book_3 = df.loc[df['ru_title']==title_3].index[0]
if liked_book_3.empty:
    print("I don`t know third book name, try to check the correctness \n")

ID_book_1

# liked_book_1 = ''.join(dfx['Lemmatized'])


110

In [301]:
text_bow_book_1 = ''.join(liked_book_1['Lemmatized'])
text_bow_book_2 = ''.join(liked_book_2['Lemmatized'])
text_bow_book_3 = ''.join(liked_book_3['Lemmatized'])
all_bow_books = []
all_bow_books.append(text_bow_book_1)
all_bow_books.append(text_bow_book_2)
all_bow_books.append(text_bow_book_3)


In [302]:
tfidf = TfidfVectorizer() #подбираем по частоте 
x_tfidf = tfidf.fit_transform(all_bow_books).toarray()
df_tfidf = pd.DataFrame(x_tfidf, columns = tfidf.get_feature_names())

persent_tfidf = 0

book_1_tfidf = tfidf.transform([text_bow_book_1])
cos_tfidf_book_1 = np.round(1 - pairwise_distances(df_tfidf, book_1_tfidf, metric= 'cosine'), 3)
book_2_tfidf = tfidf.transform([text_bow_book_2])
cos_tfidf_book_2 = np.round(1 - pairwise_distances(df_tfidf, book_2_tfidf, metric= 'cosine'), 3)
ready_tfidf = [cos_tfidf_book_1[1][0], cos_tfidf_book_1[2][0], cos_tfidf_book_2[2][0]]
if ready_tfidf[0] and ready_tfidf[1] and ready_tfidf[2] >= 0.5:
    koeff = max(ready_tfidf) - min(ready_tfidf) #Разница максимального с минимальным
    if koeff <= 0.4: persent_tfidf = 60
    if koeff <= 0.3: persent_tfidf = 70
    if koeff <= 0.2: persent_tfidf = 80
    if koeff <= 0.1: persent_tfidf = 90
else:
    persent_tfidf = (ready_tfidf[0] + ready_tfidf[1] + ready_tfidf[2])/3*0.5*100
persent_tfidf

C:\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


90

In [303]:
cv = CountVectorizer()
X = cv.fit_transform(all_bow_books).toarray()#перевели в массив
features = cv.get_feature_names() 
df_bow = pd.DataFrame(X, columns= features) #делаем мешок слов

persent_bow = 0

book_1_bow = cv.transform([text_bow_book_1]).toarray()
cos_bow_book_1 = np.round(1 - pairwise_distances(df_bow, book_1_bow, metric= 'cosine'), 3)
book_2_bow = cv.transform([text_bow_book_2]).toarray()
cos_bow_book_2 = np.round(1 - pairwise_distances(df_bow, book_2_bow, metric= 'cosine'), 3)
ready_bow = [cos_bow_book_1[1][0], cos_bow_book_1[2][0], cos_bow_book_2[2][0]]
min_bow = min(ready_bow)
if min_bow <= 0.9: persent_bow = 90
if min_bow <= 0.8: persent_bow = 80
if min_bow <= 0.7: persent_bow = 70
if min_bow <= 0.6: persent_bow = 60
if min_bow <= 0.5: persent_bow = 50
if min_bow <= 0.4: persent_bow = 40
if min_bow <= 0.3: persent_bow = 30
if min_bow <= 0.2: persent_bow = 20
if min_bow <= 0.1: persent_bow = 10
print(ready_bow)
persent_bow

[0.603, 0.696, 0.651]


C:\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


70

In [306]:
persent_tag = 0
tag_koef = 1.275
tag_book_1 = list(df_tag.iloc[ID_book_1][:])
tag_book_2 = list(df_tag.iloc[ID_book_2][:])
tag_book_3 = list(df_tag.iloc[ID_book_3][:])
i = 0
for tag in tag_book_1:
    if abs(tag_book_1[i] - tag_book_2[i]) <= tag_koef and abs(tag_book_2[i] - tag_book_3[i]) <= tag_koef and abs(tag_book_1[i] - tag_book_3[i]) <= tag_koef :
        if i < 3 and i > 5:
            persent_tag += 50/3
        else: persent_tag += 25/3
    i +=1
persent_tag

75.0

In [307]:


d2v_book_1 = list(df_d2v.iloc[ID_book_1][:])
d2v_book_2 = list(df_d2v.iloc[ID_book_2][:])
d2v_book_3 = list(df_d2v.iloc[ID_book_3][:])
cos_sim_12 = dot(d2v_book_1, d2v_book_2)/(norm(d2v_book_1)*norm(d2v_book_2))
cos_sim_13 = dot(d2v_book_1, d2v_book_3)/(norm(d2v_book_1)*norm(d2v_book_3))
cos_sim_32 = dot(d2v_book_3, d2v_book_2)/(norm(d2v_book_3)*norm(d2v_book_2))
cos_sim = [cos_sim_12, cos_sim_13, cos_sim_32]
if cos_sim[1] and cos_sim[0] and cos_sim[2] >= 0.5:
    koeff = max(cos_sim) - min(cos_sim) #Разница максимального с минимальным
    if koeff <= 0.4: persent_d2v = 60
    if koeff <= 0.3: persent_d2v = 70
    if koeff <= 0.2: persent_d2v = 80
    if koeff <= 0.1: persent_d2v = 90
else:
    persent_d2v = (cos_sim[0] + cos_sim[1] + cos_sim[2])/3*0.5*100
persent_d2v



16.976213041399543

In [308]:
sem_book_1 = list(df_semsim.iloc[ID_book_1][:])
sem_book_2 = list(df_semsim.iloc[ID_book_2][:])
sem_book_3 = list(df_semsim.iloc[ID_book_3][:])
sem_sim_books = [0, 0, 0 ]
i = 0
for sem in sem_book_1:
    if sem_book_1[i] == sem_book_2[i]: sem_sim_books[0] += 0.2
    if sem_book_3[i] == sem_book_2[i]: sem_sim_books[1] += 0.2
    if sem_book_1[i] == sem_book_3[i]: sem_sim_books[2] += 0.2   
    i +=1
persent_semsim = min(sem_sim_books) * 10
persent_semsim

0

In [309]:

# ['persent_semsim', 'persent_d2v', 'persent_tag', 'persent_bow', 'persent_tfidf']
data = {'Name': ['SemSim', 'D2V_Listed', 'Listed', 'Simil_bow' , 'sim_tfidf'], 'persent': [persent_semsim, persent_d2v, persent_tag, persent_bow, persent_tfidf]} 
accuracy_factor = pd.DataFrame(data)
accuracy_factor['Rating_Rank'] = accuracy_factor['persent'].rank(ascending = 1)
accuracy_factor

,Name,persent,Rating_Rank
0,SemSim,0.000000,1.0
1,D2V_Listed,16.976213,2.0
2,Listed,75.000000,4.0
3,Simil_bow,70.000000,3.0
4,sim_tfidf,90.000000,5.0


In [310]:
SemSim = accuracy_factor['Rating_Rank'][0]
D2V_Listed = accuracy_factor['Rating_Rank'][1]
Listed = accuracy_factor['Rating_Rank'][2]
Simil_bow = accuracy_factor['Rating_Rank'][3]
sim_tfidf = accuracy_factor['Rating_Rank'][4]


# Косинусное сходство

In [311]:
cv = CountVectorizer()
X = cv.fit_transform(df['Lemmatized']).toarray()#перевели в массив
features = cv.get_feature_names() 
df_bow = pd.DataFrame(X, columns= features) #делаем мешок слов
# df_bow.head(20)

C:\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [312]:
tfidf = TfidfVectorizer() #подбираем по частоте 
x_tfidf = tfidf.fit_transform(df['Lemmatized']).toarray()
df_tfidf = pd.DataFrame(x_tfidf, columns = tfidf.get_feature_names())
# df_tfidf.head(20)

In [313]:
cosine_value = []
Quest_bow = cv.transform([text_bow_book_1]).toarray()
cosine_value_bow_1 = np.round(1 - pairwise_distances(df_bow, Quest_bow, metric= 'cosine'), 3)
Quest_bow = cv.transform([text_bow_book_2]).toarray()
cosine_value_bow_2 = np.round(1 - pairwise_distances(df_bow, Quest_bow, metric= 'cosine'), 3) 
Quest_bow = cv.transform([text_bow_book_3]).toarray()
cosine_value_bow_3 = np.round(1 - pairwise_distances(df_bow, Quest_bow, metric= 'cosine'), 3) 
i = 0
for n in cosine_value_bow_1:
    cos_bow = np.round((cosine_value_bow_1[i][0] + cosine_value_bow_2[i][0] + cosine_value_bow_3[i][0])/3 , 3)
    cosine_value.append(cos_bow)
    i+=1
print(cosine_value)



[0.712, 0.51, 0.782, 0.582, 0.693, 0.635, 0.674, 0.703, 0.686, 0.698, 0.614, 0.695, 0.569, 0.322, 0.504, 0.515, 0.657, 0.56, 0.535, 0.437, 0.684, 0.459, 0.5, 0.61, 0.627, 0.532, 0.613, 0.59, 0.482, 0.722, 0.638, 0.656, 0.625, 0.546, 0.6, 0.595, 0.454, 0.422, 0.442, 0.459, 0.544, 0.583, 0.563, 0.496, 0.498, 0.503, 0.63, 0.528, 0.651, 0.499, 0.615, 0.57, 0.661, 0.742, 0.603, 0.647, 0.55, 0.682, 0.457, 0.561, 0.367, 0.663, 0.683, 0.598, 0.734, 0.685, 0.541, 0.496, 0.471, 0.457, 0.357, 0.528, 0.423, 0.352, 0.553, 0.628, 0.582, 0.419, 0.648, 0.621, 0.712, 0.691, 0.735, 0.706, 0.682, 0.717, 0.416, 0.592, 0.582, 0.439, 0.476, 0.476, 0.695, 0.537, 0.592, 0.52, 0.257, 0.634, 0.648, 0.526, 0.401, 0.691, 0.613, 0.596, 0.635, 0.654, 0.697, 0.479, 0.668, 0.687, 0.757, 0.591, 0.486, 0.509, 0.588, 0.562, 0.602, 0.444, 0.766, 0.519, 0.751, 0.459, 0.584]


In [314]:
df['Simil_bow'] = cosine_value

In [315]:
cos_value = []
Quest_tfidf = tfidf.transform([text_bow_book_1]).toarray()
cosine_value_tfidf_1 = np.round(1 - pairwise_distances(df_tfidf, Quest_tfidf, metric= 'cosine'), 3)
Quest_tfidf = tfidf.transform([text_bow_book_2]).toarray()
cosine_value_tfidf_2 = np.round(1 - pairwise_distances(df_tfidf, Quest_tfidf, metric= 'cosine'), 3) 
Quest_tfidf = tfidf.transform([text_bow_book_3]).toarray()
cosine_value_tfidf_3 = np.round(1 - pairwise_distances(df_tfidf, Quest_tfidf, metric= 'cosine'), 3) 
i = 0
for n in cosine_value_tfidf_1:
    cos_tfidf = np.round((cosine_value_tfidf_1[i][0] + cosine_value_tfidf_2[i][0] + cosine_value_tfidf_3[i][0])/3 , 3)
    cos_value.append(cos_tfidf)
    i+=1
print(cos_value)


[0.34, 0.192, 0.553, 0.276, 0.435, 0.269, 0.347, 0.413, 0.448, 0.352, 0.231, 0.337, 0.305, 0.133, 0.21, 0.195, 0.407, 0.261, 0.179, 0.178, 0.408, 0.203, 0.144, 0.277, 0.393, 0.223, 0.246, 0.253, 0.233, 0.412, 0.314, 0.295, 0.286, 0.223, 0.229, 0.298, 0.187, 0.163, 0.142, 0.197, 0.258, 0.255, 0.291, 0.22, 0.167, 0.191, 0.405, 0.153, 0.306, 0.238, 0.162, 0.235, 0.437, 0.437, 0.364, 0.38, 0.174, 0.324, 0.122, 0.257, 0.219, 0.391, 0.259, 0.289, 0.375, 0.299, 0.204, 0.184, 0.251, 0.209, 0.129, 0.207, 0.129, 0.118, 0.295, 0.369, 0.316, 0.184, 0.336, 0.326, 0.448, 0.399, 0.429, 0.336, 0.279, 0.395, 0.126, 0.346, 0.276, 0.148, 0.244, 0.244, 0.37, 0.328, 0.263, 0.223, 0.101, 0.242, 0.35, 0.278, 0.175, 0.36, 0.377, 0.215, 0.329, 0.3, 0.311, 0.218, 0.359, 0.323, 0.471, 0.234, 0.206, 0.218, 0.232, 0.257, 0.336, 0.154, 0.563, 0.298, 0.571, 0.213, 0.215]


In [316]:
df['sim_tfidf'] = cos_value



# Финальный подбор рекомендованных книг

In [325]:
df_results = pd.DataFrame(df, columns= ['Simil_bow','sim_tfidf','SemSim','Listed', 'D2V_Listed'])

# Создать пустой список

df_results_list =[]
d = []
  
# Итерировать по каждой строке

for i in range((df_results.shape[0])):

    df_results_list.append(list(df_results.iloc[i, :]))

answer_result = df[df['title'] == title_1]
answer_answer_result = pd.DataFrame(answer_result, columns= ['Simil_bow','sim_tfidf','SemSim','Listed', 'D2V_Listed'])
for i in range((answer_answer_result.shape[0])):
    d.append(list(answer_answer_result.iloc[i, :]))
z_results = []
for answer in d:
    for result in df_results_list:
        z = 0
        if result[0] > max(cosine_value)//3*2:
            z += Simil_bow
        if result[1] > max(cos_value)//3*2:
            z += sim_tfidf       
        if result[2] == answer[2]:
            z += SemSim
        if result[3] == answer[3]:
            z += Listed
        if result[4] == answer[4]:
            z += D2V_Listed
        z_results.append(z)
df['Z_Answer'] = pd.DataFrame(z_results)
# cos_answer = (np.round(1 - pairwise_distances(df_results_list, answer_result, metric= 'cosine'), 2)* 100)

import difflib
ratio_results = []
for results in df_results_list:
    

    matcher = difflib.SequenceMatcher(None, ''.join(str(answer_result)), ''.join(str(results))).ratio()
    ratio_results.append(np.round((matcher * 100),3) )

df['Answer'] = pd.DataFrame(ratio_results)
df_answer_sort = df.sort_values(by = 'Z_Answer', ascending= False)
df_answer_sort_clear = df_answer_sort[df_answer_sort['title'].str.contains(title_1) == False] 
df_answer_sort_clear = df_answer_sort_clear[df_answer_sort_clear['title'].str.contains(title_2) == False] 
df_answer_sort_clear = df_answer_sort_clear[df_answer_sort_clear['title'].str.contains(title_3) == False] 
# Распечатать список
df_answer_sort_clear

,title,author,genre,text,ru_title,Lemmatized,D2V_Listed,Listed,SemSim,Simil_bow,sim_tfidf,Z_Answer,Answer
84,The Four Pools Mystery,Gene Webster,"Prose, Historical",CHAPTER I — INTRODUCING TERRY PATTEN\r\n\r\nIt...,Тайна четырех бассейнов,chapter introduce forgery case first make acqu...,1,4,7,0.682,0.279,14.0,4.685
35,Three Detective Anecdotes,Charles Dickens,"Story, Romance, Detective, Psychological, Social",I. - The Pair of Gloves\r\n\r\n'It's a singler...,Три рассказа о сыщиках,pair glove singler story sir say inspector wie...,1,4,2,0.595,0.298,14.0,3.964
75,Odour of Chrysanthemums,David Herbert Lawrence,Story,"I\r\n\r\nThe small locomotive engine, Number 4...",Запах хризантем,small locomotive engine number 4 come clank st...,1,4,3,0.628,0.369,14.0,4.685
110,Dracula,Bram Stoker,Gothic,CHAPTER I\r\n\r\nJONATHAN HARKER’S JOURNAL (Ke...,Дракула,journal keep shorthand 3 may bistritz leave mu...,4,4,4,0.757,0.471,13.0,4.324
11,Little Men,Louise May Alcott,Novel,"Chapter I. Nat\r\n\r\n""Please, sir, is this Pl...",Маленькие мужчины,please sir plumfield ask ragged boy man open g...,3,4,4,0.695,0.337,13.0,4.324
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Middlemarch,George Eliot,"Realistic romance, Drama, About love, Historic...",PRELUDE\r\n\r\nWho that cares much to know the...,Мидлмарч,prelude care much know history man mysterious ...,6,9,3,0.682,0.324,8.0,4.685
54,The Scarlet Letter,Nathaniel Hawthorne,"Novel, Historical, Psychological, Realism, Soc...",THE CUSTOM-HOUSE\r\n\r\nIntroductory to The Sc...,Алая буква,custom house introductory scarlet letter littl...,6,7,3,0.603,0.364,8.0,4.685
52,24 Hours in the Life of a Woman,Stefan Zweig,"Storytelling, Romance, Drama, Psychological, R...",IN THE LITTLE GUEST HOUSE on the Riviera where...,24 часа из жизни женщины,little guest house riviera stay time war heat ...,6,1,6,0.661,0.437,8.0,5.405
49,Lady Susan,Jane Austen,"Novel, About love, Psychological, Realism, Sat...",I — LADY SUSAN VERNON TO MR. VERNON\r\n\r\nLan...,Леди Сьюзан,lady mr dec dear brother longer refuse pleasur...,6,2,3,0.499,0.238,8.0,4.324


In [318]:
answer_sort = pd.DataFrame(df_answer_sort_clear, columns= ['ru_title', 'title','author','Z_Answer'])

# Создать пустой список

answer_sort_list =[]

  
# Итерировать по каждой строке

for i in range((answer_sort.shape[0])):

  

    # Использование iloc для доступа к значениям

    # текущая строка, обозначенная "i"

    answer_sort_list.append(list(answer_sort.iloc[i, :]))
    book_1 = ['','','',0]
    book_2 = ['','','',0]
    book_3 = ['','','',0]
    num = 0
    
for answer in answer_sort_list:
    i = 0
    if num == 1:
        for name in answer:
            book_1[i] = name
            i +=1
    if num == 2:
        for name in answer:
            book_2[i] = name
            i +=1
    if num == 3:
        for name in answer:
            book_3[i] = name
            i +=1
    num += 1
    if num > 3:
        break
            
print('Literator recomends you: \n')
print(str(book_1) + '\n')
print(str(book_2) + '\n')
print(str(book_3) + '\n')
# print(book_3[0])

Literator recomends you: 

['Три рассказа о сыщиках', 'Three Detective Anecdotes', 'Charles Dickens', 14.0]

['Запах хризантем', 'Odour of Chrysanthemums', 'David Herbert Lawrence', 14.0]

['Дракула', 'Dracula', 'Bram Stoker', 13.0]



# Фильтр

In [319]:
df_forfiltre = df_answer_sort.copy(deep=True)

df_forfiltre.drop('text', axis=1, inplace=True) 
df_forfiltre.drop('Lemmatized', axis=1, inplace=True) 
df_forfiltre.drop('ru_title', axis=1, inplace=True) 
df_forfiltre.drop('sim_tfidf', axis=1, inplace=True) 
df_forfiltre.drop('Answer', axis=1, inplace=True) 
df_forfiltre.drop('D2V_Listed', axis=1, inplace=True) 
df_forfiltre.drop('Listed', axis=1, inplace=True) 
df_forfiltre.drop('SemSim', axis=1, inplace=True) 
df_forfiltre.drop('Simil_bow', axis=1, inplace=True) 
df_forfiltre

,title,author,genre,Z_Answer
118,"Tom Sawyer, Detective",Mark Twain,detective,15.0
84,The Four Pools Mystery,Gene Webster,"Prose, Historical",14.0
35,Three Detective Anecdotes,Charles Dickens,"Story, Romance, Detective, Psychological, Social",14.0
75,Odour of Chrysanthemums,David Herbert Lawrence,Story,14.0
110,Dracula,Bram Stoker,Gothic,13.0
...,...,...,...,...
57,Middlemarch,George Eliot,"Realistic romance, Drama, About love, Historic...",8.0
54,The Scarlet Letter,Nathaniel Hawthorne,"Novel, Historical, Psychological, Realism, Soc...",8.0
52,24 Hours in the Life of a Woman,Stefan Zweig,"Storytelling, Romance, Drama, Psychological, R...",8.0
49,Lady Susan,Jane Austen,"Novel, About love, Psychological, Realism, Sat...",8.0


In [320]:
print('Which style do you prefer now?')
print('Romance, Drama, Psychological, Realism, Social, Story, Horror, Gothic, Scientific literature, Ironic, About love, Storytelling, Biography and memoirs, Detective, Adventure, novel, Prose, Fantasy, About love, Historical')
yesstyle = input()
print('Which style you won`t read now?')
print('Romance, Drama, Psychological, Realism, Social, Story, Horror, Gothic, Scientific literature, Ironic, About love, Storytelling, Biography and memoirs, Detective, Adventure, novel, Prose, Fantasy, About love, Historical')
nostyle = input()

Which style do you prefer now?
Romance, Drama, Psychological, Realism, Social, Story, Horror, Gothic, Scientific literature, Ironic, About love, Storytelling, Biography and memoirs, Detective, Adventure, novel, Prose, Fantasy, About love, Historical
Which style you won`t read now?
Romance, Drama, Psychological, Realism, Social, Story, Horror, Gothic, Scientific literature, Ironic, About love, Storytelling, Biography and memoirs, Detective, Adventure, novel, Prose, Fantasy, About love, Historical


In [321]:
df_filtreno = df_forfiltre[df_forfiltre['genre'].str.contains(yesstyle) == True] 
df_filtreno

,title,author,genre,Z_Answer
24,The Signal Man,Charles Dickens,"Story, Romance, Mystic, Psychological, Horror",13.0
119,The Red Room,Herbert George Wells,Horror,13.0
121,The Unnamable,Howard Phillips Lovecraft,Horror,10.0
60,The Pit and the Pendulum,Edgar Allan Poe,"Story, Romance, Psychological, Realism, Horror...",10.0
20,The Turn of the Screw,Henry James,"Psychological, Horror, Story",9.0
120,The Horla,Guy de Maupassant,Horror,8.0
21,The Horror at Red Hook,Howard Phillips Lovecraft,"Story, Psychological, Horror",8.0
25,The Judge's House,Bram Stoker,"Story, Romance, Psychological, Horror, Fantasy",8.0
122,The Body Snatcher,Robert Louis Stevenson,Horror,8.0


In [322]:
df_filtrready = df_filtreno[df_filtreno['genre'].str.contains(nostyle) == False] 
df_filtrready
l = len(df_filtrready['genre']) 
l

9

In [323]:
df_filtrready

,title,author,genre,Z_Answer
24,The Signal Man,Charles Dickens,"Story, Romance, Mystic, Psychological, Horror",13.0
119,The Red Room,Herbert George Wells,Horror,13.0
121,The Unnamable,Howard Phillips Lovecraft,Horror,10.0
60,The Pit and the Pendulum,Edgar Allan Poe,"Story, Romance, Psychological, Realism, Horror...",10.0
20,The Turn of the Screw,Henry James,"Psychological, Horror, Story",9.0
120,The Horla,Guy de Maupassant,Horror,8.0
21,The Horror at Red Hook,Howard Phillips Lovecraft,"Story, Psychological, Horror",8.0
25,The Judge's House,Bram Stoker,"Story, Romance, Psychological, Horror, Fantasy",8.0
122,The Body Snatcher,Robert Louis Stevenson,Horror,8.0


In [324]:
show_these_orders = [(l-1), (l-2), (l-3)] #вывести последние 3 из списка)[(l-(l-2)), (l-(l-3)), (l-(l-4))] 
df_questionsto2diterations = df_filtrready.iloc[show_these_orders]#проверка по последним с конца так как они меньше всего схожи, если они понравятся то эт оповод для поиска новый зависимостей, нет, то мы собрали обратную связь для поднятия коэффициента отбора(тот который сейчас 8.000)
df_questionsto2diterations

,title,author,genre,Z_Answer
122,The Body Snatcher,Robert Louis Stevenson,Horror,8.0
25,The Judge's House,Bram Stoker,"Story, Romance, Psychological, Horror, Fantasy",8.0
21,The Horror at Red Hook,Howard Phillips Lovecraft,"Story, Psychological, Horror",8.0
